<a href="https://colab.research.google.com/github/kgautam2103/Eventbrite_automation_libraray_public/blob/main/event_date_update_public.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# when you run this code, you will get a link below
#Follow the link, copy the code, paste it in the box and then press Enter on the keyboard.
#This will authenticate you and will allow you to interact with Google Sheets and other Google apps in your notebook.
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
import pandas as pd
import requests
import json
from datetime import datetime, timedelta, timezone
from dateutil import parser
from google.colab import files

#auth token for Eventbrite account admin user
auth_token = ''
prod_url = 'https://www.eventbriteapi.com/v3/'
events= '/events/'
org = '/organizations/'
counter=1
event_details_list=[]

def event_details_list_fn(event_response_list):
  event_details_list=[]
  for event_detail in event_response_list:
    event_id = event_detail['id']
    status = event_detail['status']
    name = event_detail['name']["text"]
    start_time = event_detail['start']["utc"]
    end_time = event_detail['end']["utc"]
    timezone_value = event_detail['start']['timezone']
    is_parent = event_detail['is_series_parent']
    event_details_list.append([event_id,status,name,start_time, end_time,timezone_value,is_parent])
  return event_details_list



org_id = "502996851443"

authorization = 'Bearer ' + auth_token
auth_headers = {
    'Authorization': authorization,
    'Content-Type': 'application/json'
    }

list_event_org_url = prod_url+org+org_id+events
list_event_org_response = requests.get(list_event_org_url,headers=auth_headers)



In [ ]:
list_events_org_response_json = list_event_org_response.json()
#print(list_events_org_response_json)

if list_event_org_response.status_code == 200:
  has_more_items = list_event_org_response.json()['pagination']['has_more_items']
  if has_more_items:
      continuation = list_event_org_response.json()['pagination']['continuation']
  event_response_list = list_event_org_response.json()['events']
  event_details_list=event_details_list_fn(event_response_list)
  
  
  
  while has_more_items:
    paginated_event_details_url = prod_url+org+org_id+events+"?continuation="+continuation
    paginated_get_event_details_response = requests.get(paginated_event_details_url,headers=auth_headers)
    has_more_items = paginated_get_event_details_response.json()['pagination']['has_more_items']
    if has_more_items:
      continuation = paginated_get_event_details_response.json()['pagination']['continuation']
    paginated_event_response_list = paginated_get_event_details_response.json()['events']
    #len(paginated_order_response_list)
    event_details_list.extend(event_details_list_fn(paginated_event_response_list))

  
event_details_df = pd.DataFrame(event_details_list,columns=['event_id','status','name','start','end','timezone','is_series'])

In [ ]:
event_details_df_final = event_details_df[event_details_df.status=="live"]
#event_details_df_final= event_details_df_final[event_details_df_final.is_series=="False"]
pd.set_option('display.max_rows', None)
event_details_df_final_1_event = event_details_df_final[event_details_df_final.start=='2022-01-22T02:00:00Z']
event_details_df_final_2_event = event_details_df_final[event_details_df_final.start=='2022-01-29T02:00:00Z']
event_details_df_final

In [ ]:
event_details_df_final_1_event

In [ ]:
event_details_df_final_2_event

In [ ]:
for index, row in event_details_df_final_1_event.iterrows():
  event_id =row['event_id']
  start_time_new='2022-01-23T02:00:00Z'
  end_time_new='2022-01-23T03:00:00Z'
  timezone_value = row['timezone']
  update_request_body = {"event": {
          "start": {"timezone": timezone_value,"utc": start_time_new},
          "end": { "timezone": timezone_value,"utc": end_time_new}
      }
      }

  update_event_values = json.dumps(update_request_body)
  print(update_event_values)
  update_event_url = prod_url+events+event_id+"/"
  update_response = requests.post(update_event_url, data=update_event_values, headers=auth_headers)
  update_status = update_response.status_code
  update_response_text = update_response.text
  print(update_response_text)


{"event": {"start": {"timezone": "America/Los_Angeles", "utc": "2022-01-23T02:00:00Z"}, "end": {"timezone": "America/Los_Angeles", "utc": "2022-01-23T03:00:00Z"}}}
{"name":{"text":"Oceanside Students: Love Yourself - Meditate","html":"Oceanside Students: Love Yourself - Meditate"},"description":{"text":"Ever loved yourself? In 2022 invest in yourself with free guided meditation workshop . Find peace, strength, healing and universal identity.","html":"Ever loved yourself? In 2022 invest in yourself with free guided meditation workshop . Find peace, strength, healing and universal identity."},"url":"https://www.eventbrite.ca/e/oceanside-students-love-yourself-meditate-tickets-242188802227","start":{"timezone":"America/Los_Angeles","local":"2022-01-22T18:00:00","utc":"2022-01-23T02:00:00Z"},"end":{"timezone":"America/Los_Angeles","local":"2022-01-22T19:00:00","utc":"2022-01-23T03:00:00Z"},"organization_id":"502996851443","created":"2022-01-09T06:51:34Z","changed":"2022-01-11T01:51:31Z","p

In [ ]:
for index, row in event_details_df_final_2_event.iterrows():
  event_id =row['event_id']
  start_time_new='2022-01-30T02:00:00Z'
  end_time_new='2022-01-30T03:00:00Z'
  timezone_value = row['timezone']
  update_request_body = {"event": {
          "start": {"timezone": timezone_value,"utc": start_time_new},
          "end": { "timezone": timezone_value,"utc": end_time_new}
      }
      }

  update_event_values = json.dumps(update_request_body)
  print(update_event_values)
  update_event_url = prod_url+events+event_id+"/"
  update_response = requests.post(update_event_url, data=update_event_values, headers=auth_headers)
  update_status = update_response.status_code
  update_response_text = update_response.text
  print(update_response_text)


{"event": {"start": {"timezone": "America/Los_Angeles", "utc": "2022-01-30T02:00:00Z"}, "end": {"timezone": "America/Los_Angeles", "utc": "2022-01-30T03:00:00Z"}}}
{"name":{"text":"Oceanside Students: Love Yourself - Meditate","html":"Oceanside Students: Love Yourself - Meditate"},"description":{"text":"Ever loved yourself? In 2022 invest in yourself with free guided meditation workshop . Find peace, strength, healing and universal identity.","html":"Ever loved yourself? In 2022 invest in yourself with free guided meditation workshop . Find peace, strength, healing and universal identity."},"url":"https://www.eventbrite.ca/e/oceanside-students-love-yourself-meditate-tickets-242188812257","start":{"timezone":"America/Los_Angeles","local":"2022-01-29T18:00:00","utc":"2022-01-30T02:00:00Z"},"end":{"timezone":"America/Los_Angeles","local":"2022-01-29T19:00:00","utc":"2022-01-30T03:00:00Z"},"organization_id":"502996851443","created":"2022-01-09T06:51:34Z","changed":"2022-01-11T01:54:33Z","p